In [27]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import torch
import torch.nn as nn
import random
import time
import math
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

In [28]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [29]:
all_letters = string.ascii_letters + " .,;:'!-?)("
n_letters = len(all_letters)

MAX_LENGTH = 15

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )



def readfile(filename):
    file = open(filename, encoding='utf-8').read().replace("\n"," ")
    return file

data = readfile("/content/sample_data/Shakespere.txt")
#print(data)

X = []
y = []

for i in range(0, len(data)-MAX_LENGTH):
    X.append(data[i:i+MAX_LENGTH-1])
    y.append(data[i+1:i+MAX_LENGTH])

print("Number of Training Example: {}".format(len(X)))
print("\nA Random Example:")
q = random.randint(0,len(X))
print("Input: {}\nOutput: {}".format(X[q],y[q]))

Number of Training Example: 94198

A Random Example:
Input:  their habitat
Output: their habitati


In [30]:
int2char = dict(enumerate(all_letters))
char2int = {c : i for i, c in int2char.items()}

In [31]:
for i in range(len(X)):
    X[i] = [char2int[c] for c in X[i]]
    y[i] = [char2int[c] for c in y[i]]

print("\nA Random Example:")
q = random.randint(0,len(X))
print("Input: {}\nOutput: {}".format(X[q],y[q]))


A Random Example:
Input: [52, 8, 13, 52, 4, 21, 4, 17, 24, 52, 1, 11, 4, 18]
Output: [8, 13, 52, 4, 21, 4, 17, 24, 52, 1, 11, 4, 18, 18]


In [32]:
dict_size = len(char2int)
seq_len = MAX_LENGTH-1
batch_size = len(X)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    for i in range(batch_size):
        for u in range(seq_len):
            #print(i, u, sequence[i][u])
            features[i, u, sequence[i][u] ] = 1
    return features

In [33]:
X = one_hot_encode(X, dict_size, seq_len, batch_size)
#y = one_hot_encode(y, dict_size, seq_len, batch_size)

In [34]:
X = torch.from_numpy(X).cuda()
y = torch.tensor(y).cuda()

In [35]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers):
        super(RNN,self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, input):
        batch_size = input.size(0)
        
        hidden = self.initHidden(batch_size)
        
        out, hidden = self.rnn(input, hidden)
        
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)
        
        return out, hidden
    
    def initHidden(self, batch_size):
        return torch.zeros(self.n_layers, batch_size, self.hidden_size).cuda()

In [36]:
rnn = RNN(dict_size, 128, dict_size, n_layers=1)
rnn.cuda()
n_epochs = 2000
lr = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [37]:
for epoch in range(1, n_epochs+1):
    optimizer.zero_grad()
    X.cuda()
    y.cuda()
    output, hidden = rnn(X)
    #print(output)
    loss = criterion( output, y.view(-1).long() )
    #print(loss)
    loss.backward()
    optimizer.step()
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/2000............. Loss: 3.5079
Epoch: 20/2000............. Loss: 3.1515
Epoch: 30/2000............. Loss: 3.1007
Epoch: 40/2000............. Loss: 3.0791
Epoch: 50/2000............. Loss: 3.0638
Epoch: 60/2000............. Loss: 3.0493
Epoch: 70/2000............. Loss: 3.0333
Epoch: 80/2000............. Loss: 3.0137
Epoch: 90/2000............. Loss: 2.9898
Epoch: 100/2000............. Loss: 2.9580
Epoch: 110/2000............. Loss: 2.9168
Epoch: 120/2000............. Loss: 2.8659
Epoch: 130/2000............. Loss: 2.8141
Epoch: 140/2000............. Loss: 2.7625
Epoch: 150/2000............. Loss: 2.7126
Epoch: 160/2000............. Loss: 2.6658
Epoch: 170/2000............. Loss: 2.6186
Epoch: 180/2000............. Loss: 2.5748
Epoch: 190/2000............. Loss: 2.5331
Epoch: 200/2000............. Loss: 2.4947
Epoch: 210/2000............. Loss: 2.4593
Epoch: 220/2000............. Loss: 2.4269
Epoch: 230/2000............. Loss: 2.3989
Epoch: 240/2000............. Loss: 2.3712
E

In [38]:
torch.save(rnn, 'shakespeare-rnn-generation.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [39]:
def predict(model, character):
    model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)

    character = torch.from_numpy(character).cuda()
    
    out, hidden = model(character)
    out
    prob = nn.functional.softmax(out[-1], dim=0).data
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [40]:
def sample(model, out_len, start):
    model
    model.eval() # eval mode
    start = start.lower()
    chars = [ch for ch in start]
    size = out_len - len(chars)
    for ii in range(size):
        char, h = predict(model, chars)
        # if char == '-':
        #     char="\n"
        chars.append(char)

    return ''.join(chars)

In [50]:
sample(rnn, 200, "Winter")

'winter that the world when the world when the world when the world when the world when the world when the world when the world when the world when the world when the world when the world when the worl'